<a href="https://colab.research.google.com/github/DiabeticRetinopathy114/114_DR/blob/main/2015%20Data%20Colored%20Resized/CNN/2015_train_stage1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sovitrath_diabetic_retinopathy_2015_data_colored_resized_path = kagglehub.dataset_download('sovitrath/diabetic-retinopathy-2015-data-colored-resized')

print('Data source import complete.')


In [ ]:
from tensorflow import lite
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import random, os
import shutil
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy
from sklearn.model_selection import train_test_split

2025-05-14 08:43:32.788539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747212213.012903      77 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747212213.088210      77 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Add an additional column, mapping to the type
df = pd.read_csv(r'../input/diabetic-retinopathy-2015-data-colored-resized/trainLabels.csv')

diagnosis_dict_binary = {
    0: 'No_DR',
    1: 'DR',
    2: 'DR',
    3: 'DR',
    4: 'DR'
}

diagnosis_dict = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}


df['binary_type'] =  df['level'].map(diagnosis_dict_binary.get)
df['type'] = df['level'].map(diagnosis_dict.get)
df.head()

,image,level,binary_type,type
0,10_left,0,No_DR,No_DR
1,10_right,0,No_DR,No_DR
2,13_left,0,No_DR,No_DR
3,13_right,0,No_DR,No_DR
4,15_left,1,DR,Mild


**資料前處理：建立兩個資料集**

In [ ]:
# 假設 train_df 是包含圖像路徑與原始標籤的 DataFrame
df['binary_type'] = df['level'].apply(lambda x: 'no_DR' if x == 0 else 'DR')
df['str_label'] = df['type'].astype(str)  # 方便分類用

In [ ]:
# 拆分成兩個資料集
df_stage1 = df.copy()
df_stage2 = df[df['level'] != 0].copy()
df_stage2['stage2_label'] = df_stage2['level'] - 1
df_stage2['image_path'] = df_stage2.apply(lambda row: f"{row['type']}/{row['image']}.png", axis=1)  # 將 1~4 → 0~3（方便分類）

 **建立資料產生器（ImageDataGenerator**

In [ ]:
# --- Stage 1: 無DR vs 有DR（二元分類） ---

from tensorflow.keras.preprocessing.image import ImageDataGenerator

df['image_path'] = df.apply(lambda row: f"{row['type']}/{row['image']}.png", axis=1)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory='../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images',
    x_col='image_path',
    y_col='binary_type',  # 必須是 'no_DR' / 'DR'
    class_mode='binary',
    target_size=(224, 224),
    subset='training'
)

val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory='../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images',
    x_col='image_path',
    y_col='binary_type',  # 必須是 'no_DR' / 'DR'
    class_mode='binary',
    target_size=(224, 224),
    subset='validation'
)

Found 28101 validated image filenames belonging to 2 classes.
Found 7025 validated image filenames belonging to 2 classes.


In [ ]:
import os, shutil

# 假設圖片來自這個結構
src_root = '../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images'
diagnosis_dict = {0: 'No_DR', 1: 'Mild', 2: 'Moderate', 3: 'Severe', 4: 'Proliferate_DR'}

# 創建 Stage 2 資料夾
dst_root = 'train_stage2'
if os.path.exists(dst_root):
    shutil.rmtree(dst_root)
os.makedirs(dst_root)

# # 過濾出 DR 圖（level ≠ 0）
# df_stage2 = df[df['level'] != 0].copy()
# df_stage2['stage2_label'] = df_stage2['level'] - 1

# 建立子資料夾與搬圖
for _, row in df_stage2.iterrows():
    level = row['level']
    class_folder = str(level - 1)  # 1~4 → 0~3
    filename = row['image'] + '.png'
    folder_name = diagnosis_dict[level]

    srcfile = os.path.join(src_root, folder_name, filename)
    dst_dir = os.path.join(dst_root, class_folder)
    os.makedirs(dst_dir, exist_ok=True)

    if os.path.exists(srcfile):
        shutil.copy(srcfile, dst_dir)
    else:
        print(f'❌ 找不到圖片：{srcfile}')

In [ ]:
# --- Stage 2: 多分類（4類） ---

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_stage2_gen = datagen.flow_from_directory(
    'train_stage2',
    target_size=(224, 224),
    class_mode='sparse', #籤輸出為整數類別 ID（例如 0, 1, 2, 3）
    subset='training'
)

val_stage2_gen = datagen.flow_from_directory(
    'train_stage2',
    target_size=(224, 224),
    class_mode='sparse',
    subset='validation'
)

Found 7455 images belonging to 4 classes.
Found 1861 images belonging to 4 classes.


**第一階段模型（二元分類）**

In [ ]:
model_stage1 = tf.keras.Sequential([
    layers.Conv2D(16, (3,3), padding="same", activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(32, (3,3), padding="same", dilation_rate=2, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), padding="same", dilation_rate=2, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(1, activation='sigmoid')
])

# model_v3.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
#                  loss=tf.keras.losses.BinaryCrossentropy(),
#                  metrics=['accuracy'])

model_stage1.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

history = model_stage1.fit(train_gen,
                 epochs=50,
                 validation_data=val_gen)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747212805.020217      77 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1747212811.210822     146 service.cc:148] XLA service 0x7ed0c8004ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747212811.214125     146 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747212811.759927     146 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/879 ━━━━━━━━━━━━━━━━━━━━ 1:08 78ms/step - accuracy: 0.6337 - loss: 2.8690  

I0000 00:00:1747212816.714139     146 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


879/879 ━━━━━━━━━━━━━━━━━━━━ 168s 181ms/step - accuracy: 0.7143 - loss: 1.0131 - val_accuracy: 0.7305 - val_loss: 0.5784
Epoch 2/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 92s 104ms/step - accuracy: 0.7380 - loss: 0.5812 - val_accuracy: 0.7305 - val_loss: 0.6341
Epoch 3/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 93s 105ms/step - accuracy: 0.7326 - loss: 0.5811 - val_accuracy: 0.7305 - val_loss: 0.5789
Epoch 4/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 92s 104ms/step - accuracy: 0.7400 - loss: 0.5678 - val_accuracy: 0.7305 - val_loss: 0.5734
Epoch 5/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 92s 104ms/step - accuracy: 0.7354 - loss: 0.5808 - val_accuracy: 0.7305 - val_loss: 0.5787
Epoch 6/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 88s 100ms/step - accuracy: 0.7360 - loss: 0.5770 - val_accuracy: 0.7305 - val_loss: 0.5828
Epoch 7/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 87s 99ms/step - accuracy: 0.7366 - loss: 0.5766 - val_accuracy: 0.7304 - val_loss: 0.5830
Epoch 8/50
879/879 ━━━━━━━━━━━━━━━━━━━━ 88s 100ms/step - accuracy: 0.7415 - loss: 0.5765 - val

**第二階段模型（多分類 4 類）**

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# 從 train generator 取出所有類別標籤
y_train_stage2 = train_stage2_gen.classes  # 對應 0 ~ 3 的整數標籤

# 類別總數
n_classes = np.unique(y_train_stage2)

# 計算權重
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=n_classes,
    y=y_train_stage2
)

# 轉為 dict 格式
class_weights = {i: w for i, w in zip(n_classes, class_weights_array)}
print("📊 class_weights:", class_weights)

📊 class_weights: {0: 0.9533248081841432, 1: 0.4401865847897969, 2: 2.6663090128755367, 3: 3.287037037037037}


In [ ]:
model_stage2 = tf.keras.Sequential([
    layers.Conv2D(16, (3,3), padding="same", activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(32, (3,3), padding="same", dilation_rate=2, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), padding="same", dilation_rate=2, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(4, activation='softmax')
])

model_stage2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

history = model_stage2.fit(
    train_stage2_gen,
    epochs=100,
    validation_data=val_stage2_gen,
    class_weight=class_weights
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 29s 96ms/step - accuracy: 0.2557 - loss: 1.6099 - val_accuracy: 0.0978 - val_loss: 1.5515
Epoch 2/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step - accuracy: 0.3178 - loss: 1.3328 - val_accuracy: 0.1838 - val_loss: 1.4345
Epoch 3/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 18s 78ms/step - accuracy: 0.3284 - loss: 1.2900 - val_accuracy: 0.2370 - val_loss: 1.4209
Epoch 4/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step - accuracy: 0.3343 - loss: 1.2773 - val_accuracy: 0.2681 - val_loss: 1.3763
Epoch 5/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.3226 - loss: 1.2781 - val_accuracy: 0.3579 - val_loss: 1.3326
Epoch 6/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 18s 79ms/step - accuracy: 0.3580 - loss: 1.2393 - val_accuracy: 0.3041 - val_loss: 1.3557
Epoch 7/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 17s 72ms/step - accuracy: 0.3547 - loss: 1.2221 - val_accuracy: 0.3418 - val_loss: 1.3359
Epoch 8/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step - accuracy: 0.3924 - loss: 1

In [ ]:
# Plot Training & Validation Accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training & Validation Accuracy')

# Plot Training & Validation Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training & Validation Loss')

plt.show()

NameError: name 'plt' is not defined

In [ ]:
# loss, acc = model_stage2.evaluate(test_batches, verbose=1)
# # print("Loss: ", loss)
# print("Accuracy: ", acc)

**預測階段邏輯**

當要預測一張新圖：
1. 用 model_stage1 判斷是否為 DR。
2. 若是 DR，再用 model_stage2 預測 1~4 的等級。

In [ ]:
def predict_image(image_path, model_stage1, model_stage2):
    # 你的預測邏輯寫在這裡，例如：

    import cv2
    import numpy as np

    # 預處理圖片
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Stage 1 預測：判斷是否有 DR
    pred_stage1 = model_stage1.predict(img)
    has_dr = np.argmax(pred_stage1) != 0  # 或 prob > 0.5（看你的設計）

    if not has_dr:
        return 'DR Level: 0 (No_DR)'

    # Stage 2 預測：多分類 1~4 等級
    pred_stage2 = model_stage2.predict(img)
    stage2_class = np.argmax(pred_stage2) + 1  # 因為你訓練時是 0~3 → 要 +1

    return f'DR Level: {stage2_class}'

In [ ]:
predict_image('../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Proliferate_DR/10321_left.png', model_stage1, model_stage2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step


'DR Level: 0 (No_DR)'